In [1]:
import pandas as pd
import numpy as np
import random
import joblib
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize, LabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report 
from sklearn.metrics import roc_curve, auc

In [2]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [3]:
#def sample_train_set(x, y, sample_num):
#    idx_list = random.sample(list(x.index), sample_num)
#    sampled_x = x.loc[idx_list]
#    sampled_y = y.loc[idx_list]
#    return np.array(sampled_x), np.array(sampled_y)

#### The Datasets

In [17]:
train_df = pd.read_csv('../../final/data/train_set_noisy_6_missing_3.csv')
train_df = train_df.fillna(-1)
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,9.401651,-1.000000,-1.000000,0.000000,116.415615,103.966653,-1.000000,2.712885,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,7.029873,102.717562,2.079564,-1.000000,-1.000000,79.373697,212.817785,2.657004,1,0.675576,104.688780,50.452242,-1.000000,-1.000000,49.832026,21.089620,-1.000000
2,15.900836,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,0.999932,90.916733,60.845387,77.678965,1.513323,139.638756,47.702509,-1.000000
3,12.710083,437.896009,-1.000000,-1.000000,392.155888,103.338649,91.089902,3.689834,1,1.431816,31.576192,124.321572,79.407308,24.911028,84.817826,-1.000000,-1.000000
4,9.256332,267.393719,-1.000000,5.758205,258.911569,76.491274,189.160039,-1.000000,1,1.259486,-1.000000,-1.000000,55.681222,-1.000000,-1.000000,27.768995,73.059709


In [18]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]
X_test.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000


#### Some useful functions and variables

In [19]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((56000, 17), (14000, 17), (56000,), (14000,))

#### Decision Tree

In [20]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)
test_df_dt = pd.DataFrame()
test_df_dt['y_actual'] = y_test
test_df_dt['y_pred'] = y_pred_dt
test_df_dt.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [21]:
success_rate_dt, success_df_dt = utils.success_rate(test_df_dt)
success_rate_dt

74.32142857142857

In [22]:
acc_dt, f1_dt, roc_auc_dt = utils.test(test_df_dt['y_actual'], test_df_dt['y_pred'])
acc_dt, f1_dt, roc_auc_dt

(0.7432142857142857, 0.7190537887675423, 0.8435118407114264)

#### Random Forest Classifier

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
test_df_rf = pd.DataFrame()
test_df_rf['y_actual'] = y_test
test_df_rf['y_pred'] = y_pred_rf
test_df_rf.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [24]:
success_rate_rf, success_df_rf = utils.success_rate(test_df_rf)
success_rate_rf

90.28571428571428

In [25]:
acc_rf, f1_rf, roc_auc_rf = utils.test(test_df_rf['y_actual'], test_df_rf['y_pred'])
acc_rf, f1_rf, roc_auc_rf

(0.9028571428571428, 0.8510546112810196, 0.931949214102628)

#### XGBoost

In [26]:
# y_train_xgb = numerize_labels(y_train)
# y_test_xgb = numerize_labels(y_test)

In [27]:
import xgboost as xgb
xg = xgb.XGBClassifier(random_state=constants.SEED).fit(X_train, y_train)
y_pred_xg = xg.predict(X_test)
test_df_xg = pd.DataFrame()
test_df_xg['y_actual'] = y_test
test_df_xg['y_pred'] = y_pred_xg
test_df_xg.isna().sum()

y_actual    0
y_pred      0
dtype: int64

In [28]:
success_rate_xg, success_df_xg = utils.success_rate(test_df_xg)
success_rate_xg

90.05

In [29]:
acc_xg, f1_xg, roc_auc_xg = utils.test(test_df_xg['y_actual'], test_df_xg['y_pred'])
acc_xg, f1_xg, roc_auc_xg

(0.9005, 0.8474436769995559, 0.9304408146605617)

#### Testing Rf and data with random zeros

In [ ]:
# #data with random zeros
# X_train = np.loadtxt('data/zeros/X_train.txt', dtype=np.float32)
# #X_val = np.loadtxt('data/zeros/X_val.txt', dtype=np.float32)
# X_test = np.loadtxt('data/zeros/X_test.txt', dtype=np.float32)

# y_train = np.loadtxt('data/zeros/y_train.txt', dtype=int)
# #y_val = np.loadtxt('data/zeros/y_val.txt', dtype=int)
# y_test = np.loadtxt('data/zeros/y_test.txt', dtype=int)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier(random_state=42).fit(X_train, y_train)
# acc, f1, cr, cm, roc_auc, roc_auc2, y_pred  = test(rf, X_test, y_test) 
# print(f'Accuracy - {acc}, F1 Score Macro: {f1}, ROC AUC Score: {roc_auc}, ROC AUC 2: {roc_auc2}')
# print(f'Unique predicted classes: {np.unique(y_pred)}')